In [2]:
import torch
import os
import random
import matplotlib.pyplot as plt
import numpy as np
from torch import nn

# 直接加载模型
model = torch.load(r'C:\Users\qishe\Desktop\ppt\epoch-20.pth')
criterion1 = nn.MSELoss()

# 将模型加载到设备上
model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# 切换到评估模式
model.eval()



# 指定路径
folder_path = r'D:\Ultra-low-field-MRI-Scanner-main\data\dataset1\data\train'

# 获取文件夹中所有 .pth 文件
pth_files = [f for f in os.listdir(folder_path) if f.endswith('.pth')]




In [3]:
# 随机选择一个 .pth 文件
random_pth_file = random.choice(pth_files)
file_path = os.path.join(folder_path, random_pth_file)
    
# 加载 .pth 文件中的字典
data_dict = torch.load(file_path)

Kspace_data = data_dict['k-space']
# 提取 'label' 数据
label_data = data_dict['label']


# 确定是否有可用的 GPU，如果有，则使用 GPU，否则使用 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 假设 model 已经被加载到 GPU 上
Kspace_data = Kspace_data.to(device)  # 将输入数据移动到 GPU

# 进行推理
with torch.no_grad():  # 禁用梯度计算，节省显存
    output = model(Kspace_data)
    

label_data = label_data.unsqueeze(-1)  # 添加一个维度到最后，使其形状变为 (1, 2, 128, 1)

output = output.to(device)
label_data = label_data.to(device)
loss = criterion1(output,label_data)



In [4]:
import torch

# 假设 Kspace_data 的形状是 [batch_size, 2, height, width]
# 通道 0 表示实部，通道 1 表示虚部

real_part = Kspace_data[:, 0, :, :]  # 提取实部
imag_part = Kspace_data[:, 1, :, :]  # 提取虚部

# 合并实部和虚部为复数张量
complex_Kspace_data = torch.complex(real_part, imag_part)
complex_Kspace_data = complex_Kspace_data.squeeze()
real_part = label_data[:, 0, :, :]  # 提取实部
imag_part = label_data[:, 1, :, :]  # 提取虚部

complex_label_data = torch.complex(real_part, imag_part)
complex_label_data = complex_label_data.squeeze(0)
complex_Kspace_data.shape

torch.Size([128, 10])

In [5]:
import torch

def complex_correlation_with_label(signals, label_signal):
    # 获取信号的数量
    num_signals = signals.size(1)
    
    # 初始化相关性数组
    correlations = torch.zeros(num_signals, dtype=torch.cfloat)

    # 计算每个信号与标签信号的相关性
    for i in range(num_signals):
        correlations[i] = torch.mean(signals[:, i] * torch.conj(label_signal[:, 0]))
    
    return correlations


# 计算相关性
correlations = complex_correlation_with_label(complex_Kspace_data, complex_label_data)
print("复数信号与标签信号的相关性:\n", correlations)

magnitudes = torch.abs(correlations)
print(magnitudes)

def rank_tensor(tensor):
    # 获取张量的排序索引，从大到小排列
    sorted_indices = torch.argsort(tensor, descending=True)
    
    # 创建一个与输入张量大小相同的排名张量
    ranks = torch.empty_like(tensor, dtype=torch.long)
    
    # 遍历排序后的索引，将对应位置的排名从1到10赋值
    for rank, idx in enumerate(sorted_indices, 1):
        ranks[idx] = rank
    
    return ranks

rank = rank_tensor(magnitudes)
# 获取根据模值排序的信号编号

# 输出信号编号
print("根据相关性的模值排序后的信号编号（从1到10）:\n", rank)



复数信号与标签信号的相关性:
 tensor([ -32.2825-18.9779j,   12.1979-22.9881j,  963.3839-760.9211j,
        -270.7272-116.0464j, 2661.4849-1200.0637j, -278.7265-89.6993j,
         679.3990+126.1604j, 3020.7212-2056.5806j,  237.6805+350.6705j,
         119.4312-131.8776j])
tensor([  37.4476,   26.0239, 1227.6438,  294.5505, 2919.5298,  292.8044,
         691.0134, 3654.3508,  423.6293,  177.9200])
根据相关性的模值排序后的信号编号（从1到10）:
 tensor([ 9, 10,  3,  6,  2,  7,  4,  1,  5,  8])


In [6]:
rank_list = []
magnitudes_list = []

for filename in os.listdir(folder_path):
     # 检查文件是否以.pth结尾
    file_path = os.path.join(folder_path, filename)

    data_dict = torch.load(file_path)
    Kspace_data = data_dict['k-space']
     # 提取 'label' 数据
    label_data = data_dict['label']
    label_data = label_data.unsqueeze(-1)  # 添加一个维度到最后，使其形状变为 (1, 2, 128, 1)
    real_part = Kspace_data[:, 0, :, :]  # 提取实部
    imag_part = Kspace_data[:, 1, :, :]  # 提取虚部

    # 合并实部和虚部为复数张量
    complex_Kspace_data = torch.complex(real_part, imag_part)
    complex_Kspace_data = complex_Kspace_data.squeeze()
    real_part = label_data[:, 0, :, :]  # 提取实部
    imag_part = label_data[:, 1, :, :]  # 提取虚部

    complex_label_data = torch.complex(real_part, imag_part)
    complex_label_data = complex_label_data.squeeze(0)
    correlations = complex_correlation_with_label(complex_Kspace_data, complex_label_data)
    magnitudes = torch.abs(correlations)


    magnitudes_list.append(magnitudes)
    rank = rank_tensor(magnitudes)
    rank_list.append(rank)

magnitudes_all = torch.stack(magnitudes_list)
rank_all = torch.stack(rank_list)

print(rank_all.shape) 



    

torch.Size([6854, 10])


In [51]:
import torch

# 假设你的张量是这样生成的

# 获取列数
num_columns = rank_all.shape[1]

for col in range(num_columns):
    # 获取当前列
    current_column = rank_all[:, col]
    
    # 获取唯一值及其计数
    unique_values, counts = torch.unique(current_column, return_counts=True)

    # 检查 counts 的大小
    if counts.size(0) < 2:
        # 如果少于两个唯一值，则打印所有唯一值及其计数
        print(f"Coil {col+1}: Unique values are {unique_values.tolist()} with counts {counts.tolist()}")
    else:
        # 获取出现次数最多的前两个值
        top_k_counts, top_k_indices = torch.topk(counts, 2)

        # 获取对应的值
        top_k_values = unique_values[top_k_indices]

        # 打印结果
        print(f"Coil {col+1}: Top 2 values are {top_k_values.tolist()} with counts {top_k_counts.tolist()}")


Coil 1: Top 2 values are [10, 9] with counts [4646, 2208]
Coil 2: Top 2 values are [9, 10] with counts [4646, 2208]
Coil 3: Unique values are [3] with counts [6854]
Coil 4: Top 2 values are [7, 6] with counts [5535, 1319]
Coil 5: Unique values are [2] with counts [6854]
Coil 6: Top 2 values are [6, 7] with counts [5535, 1319]
Coil 7: Unique values are [4] with counts [6854]
Coil 8: Unique values are [1] with counts [6854]
Coil 9: Unique values are [5] with counts [6854]
Coil 10: Unique values are [8] with counts [6854]
